In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import tree
import os
import xlrd
import pandas as pd
import seaborn
import csv
from csv import reader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib.lines import Line2D
from scipy.stats import pearsonr
import pickle
RF_SEED = 30

In [5]:
# Read the csv file into a pandas DataFrame

Property = pd.read_csv('Resources/Property.csv')
Property.head()

C:\Users\coset\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Parcel_ID,Parcel_Acreage,Property_Street_Address,Property_City,Property_State,Property_Zip,Parcel_Vacancy,Conveyance_Date,Property_Sales_Price,Seller_City,...,Buyer_ZIP,Primary_Residence,Assessed_Value_Land,Assessed_Value_Improvement,Total_Assessed_Value,Property_Class_Code,Class_Code_General,Class_Code_Translated,County_ID,County_Name
0,4574368,0.169,3909 MARIAH DRIVE ...,NEW ALBANY,IN,41750,N,6/2/2020,179900.0,NEW ALBANY,...,47150,Y,43800.0,138500.0,182300.0,510.0,500,Residential,22,Floyd
1,4515880,0.134,4110 E 10TH STREET,INDIAAPOLIS,IN,42601,N,10/19/2020,48000.0,Petaluma,...,46074,N,5000.0,88500.0,93500.0,510.0,500,Residential,49,Marion
2,4454504,0.172,5404 OAK HARBOR CT,INDIANAPOLIS,IN,42637,N,6/18/2020,195000.0,Indianapolis,...,46237,Y,30000.0,158000.0,188000.0,520.0,500,Residential,49,Marion
3,4421719,NaN,6516 EMERALD HILL COURT #310,INDIANAPOLIS,IN,42637,N,5/1/2020,120000.0,Franklin,...,42637,Y,10400.0,77900.0,88300.0,550.0,500,Residential,49,Marion
4,4531964,2.155,6335 Intech Commons Drive,Indianapolis,IN,42678,N,12/16/2020,1785000.0,MIAMI BEACH,...,75093,N,845000.0,957900.0,1802900.0,425.0,400,Commercial,49,Marion


In [6]:
len(Property)

184189

In [7]:
#Drop NA's from DF
pd.DataFrame(Property).dropna()
Property.head()
Property_df = Property[~Property.isin([np.nan, np.inf, -np.inf]).any(1)]
len(Property_df)

180360

In [8]:
#Select 5 Columns of DF
#Property_df=Property_df[['Property_Sales_Price','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy','Class_Code_Translated']]
Property_df=Property_df[['Property_Sales_Price','Class_Code_Translated','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy', 'County_Name']]

In [9]:
num_col = ['Property_Sales_Price','Total_Assessed_Value', 'Parcel_Acreage']

In [10]:

# outliers
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) > 3)].shape)
# good data
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].shape)

(646, 6)
(179715, 6)


In [11]:
#Git Rid of Outliers
Property_df = Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].reset_index()
Property_df.head()


,index,Property_Sales_Price,Class_Code_Translated,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy,County_Name
0,0,179900.0,Residential,182300.0,0.169,N,Floyd
1,1,48000.0,Residential,93500.0,0.134,N,Marion
2,2,195000.0,Residential,188000.0,0.172,N,Marion
3,4,1785000.0,Commercial,1802900.0,2.155,N,Marion
4,5,257575.0,Residential,210900.0,0.201,N,Marion


In [47]:
#One Hot encoding for non Numerical 
Property_df = pd.get_dummies(data=Property_df, columns=['Parcel_Vacancy','Class_Code_Translated','County_Name'])
#new_df.head()


KeyError: "None of [Index(['Parcel_Vacancy', 'Class_Code_Translated', 'County_Name'], dtype='object')] are in the [columns]"

In [13]:
#Git Rid of Index
Property_df.drop(['index'],axis=1,inplace=True)



In [14]:
Property_df = Property_df.loc[Property_df['Total_Assessed_Value']>0,:]
df = Property_df.loc[Property_df['Parcel_Acreage']>0,:]
#len(new_df)


## Model Build

In [33]:

#Seperate X and Y
y_data = df['Property_Sales_Price']

x_data = df.loc[:, Property_df.columns != 'Property_Sales_Price']

In [34]:
    data = df.loc[:, df.columns !='Property_Sales_Price'] 
    print( df['Property_Sales_Price'], data, data.columns )


0          179900.0
1           48000.0
2          195000.0
3         1785000.0
4          257575.0
            ...    
179710      90000.0
179711      78000.0
179712      11250.0
179713     200000.0
179714     184000.0
Name: Property_Sales_Price, Length: 171928, dtype: float64         Total_Assessed_Value  Parcel_Acreage  Parcel_Vacancy_N  \
0                   182300.0           0.169                 1   
1                    93500.0           0.134                 1   
2                   188000.0           0.172                 1   
3                  1802900.0           2.155                 1   
4                   210900.0           0.201                 1   
...                      ...             ...               ...   
179710               64500.0        4000.000                 1   
179711               46700.0        2000.000                 1   
179712               67200.0        2700.000                 1   
179713                9200.0        1564.000                 

In [35]:
feature_names = data.columns

In [43]:
def split_data_train_model(labels, data):
    # 20% examples in test data
    train, test, train_labels, test_labels = train_test_split(data,
                                                              labels,
                                                              test_size=0.2,
                                                              random_state=RF_SEED)

# training data fit
    regressor = RandomForestRegressor(n_estimators = 1000, random_state = RF_SEED)
    regressor.fit(train, train_labels)
 
    return test, test_labels, regressor

In [44]:
x_test, x_test_labels, regressor = split_data_train_model(y_data, x_data)
 
predictions = regressor.predict(x_test)

In [45]:
# find the correlation between real answer and prediction
correlation = round(pearsonr(predictions, x_test_labels)[0], 5)
 

title_name = "Random Forest Regression - Property Price vs - correlation ({})".format(correlation)
x_axis_label = "Real House Price"
y_axis_label = "Predicted House Price"
 
# plot data
#simple_scatter_plot(x_test_labels, predictions, title_name, x_axis_label, y_axis_label)

print(correlation)

0.79068


In [46]:
features_importance = regressor.feature_importances_
 
print("Feature ranking:")
for i, data_class in enumerate(feature_names):
    print("{}. {} ({})".format(i + 1, data_class, features_importance[i]))


Feature ranking:
1. Total_Assessed_Value (0.6491825351741952)
2. Parcel_Acreage (0.14663858330869195)
3. Parcel_Vacancy_N (0.015141772618275354)
4. Parcel_Vacancy_Y (0.016700170350763963)
5. Class_Code_Translated_Agricultural (0.002524415812214859)
6. Class_Code_Translated_Commercial (0.013078085544542909)
7. Class_Code_Translated_Exempt (0.011064429134688948)
8. Class_Code_Translated_Industrial (0.011674943717703291)
9. Class_Code_Translated_Locally Assessed (8.779080903282581e-05)
10. Class_Code_Translated_Mineral (1.118724273312944e-07)
11. Class_Code_Translated_Residential (0.012871622064979682)
12. County_Name_Allen (0.005793336634473317)
13. County_Name_Bartholomew (0.00040543787803269237)
14. County_Name_Benton (1.7986549685053467e-05)
15. County_Name_Blackford (2.6788113444614147e-05)
16. County_Name_Boone (0.010798741750864534)
17. County_Name_Brown (0.0002479567278459378)
18. County_Name_Carroll (3.533130187558316e-05)
19. County_Name_Cass (0.00021572778265076844)
20. County_

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk

In [ ]:
file = 'model'
outfile = open(model, 'wb')

In [ ]:
pickle.dump(feature_importance, outfile)
outfile.close()

In [ ]:
import pickle


infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [ ]:
print(new_dict)
print(new_dict==dogs_dict)
print(type(new_dict))